In [1]:
import pandas as pd
import numpy as np
import blp
from blp import blp
import xbbg
from xbbg import blp as blp_1

import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

bquery = blp.BlpQuery().start()

In [4]:
class msg1_api:
    
    '''
    Class of functions to retrieve data from MSG1.
       
    '''
        
    @staticmethod
    def get_runs(date, start_time, end_time, isin):
        
        '''
        Return RUNs for a given isin, date range and time range.
        
        date: string in format "yyyy-mm-dd"
        start_time, end_time: strings in format "Thh:mm:ss" (end_time > start_time)
        isin: string
        
        Note: UTC time is assumed, United Kingdom is UTC+01:00. For example, if you want a run for 15:00 UK time, you must
        specify the time as 14:00 UTC.
        
        '''
                       
        return bquery.bdit(isin + " @MSG1 Corp", 
                    
                    ["BID", "ASK"],

                    date + start_time,

                    date + end_time,

                    options = {"includeBrokerCodes" : True,
                               
                               "includeSpreadPrice" : True
                               }
                    )
    
    @staticmethod
    def clean_runs(dataframe):
            
        '''
        Remove empty spaces from brokercode, collapse brokerBuyCode and brokerSellCode to a single column, drop RUNs
        with size of zero.
        
        dataframe: call 'get_runs()' function as your input.
        
        '''

        temp = dataframe.copy()

        temp = temp[temp["size"] > 0]
        
        if "brokerBuyCode" not in temp.columns or "brokerSellCode" not in temp.columns:
            raise KeyError("Columns 'brokerBuyCode' and/or 'brokerSellCode' not found in the dataframe")

        temp["brokerBuyCode"] = temp["brokerBuyCode"].str.lstrip()
        temp["brokerBuyCode"] = temp["brokerBuyCode"].str.rstrip()
        temp["brokerSellCode"] = temp["brokerSellCode"].str.lstrip()
        temp["brokerSellCode"] = temp["brokerSellCode"].str.rstrip()

        temp = temp.fillna(value = "")

        temp["Dealer"] = temp["brokerBuyCode"] + temp["brokerSellCode"]

        temp.drop(["brokerBuyCode", "brokerSellCode"], axis = 1, inplace = True)

        temp.reset_index(drop = True, inplace = True)
        
        return temp
   
    @staticmethod
    def last_runs(dataframe):
        
        '''
        Return axes size, weighted average spread, number of unique dealers and top 5 dealers by size.
        
        dataframe: call 'clean_runs(get_runs())' as your input
        
        '''
        temp = dataframe.copy()
        
        index = temp.groupby(["Dealer", "type"])["time"].max()
        
        temp = pd.merge(index, temp, on = ['Dealer', "type", 'time'], how = 'left')
        
        dealer = []
        
        bid_size  = []
        
        ask_size = []
        
        bid_spd = []
        
        ask_spd = []
        
        for dlr in set(temp["Dealer"]):
            
            dealer.append(dlr)
            
            bid_subframe = temp[(temp["Dealer"] == dlr) & (temp["type"] == "BID")]
            ask_subframe = temp[(temp["Dealer"] == dlr) & (temp["type"] == "ASK")]

            bid_size.append(bid_subframe["size"].sum() if not bid_subframe.empty else 0)
            ask_size.append(ask_subframe["size"].sum() if not ask_subframe.empty else 0)

            bid_spd.append(int(bid_subframe["spreadPrice"].mean()) if not bid_subframe.empty else 0)
            ask_spd.append(int(ask_subframe["spreadPrice"].mean()) if not ask_subframe.empty else 0)

        temp = pd.DataFrame({"Dealer" : dealer, 
                             "Bid Size" : bid_size, 
                             "Ask Size" : ask_size, 
                             "Bid Spd" : bid_spd, 
                             "Ask Spd" : ask_spd
                            }
                           )
       
        return temp
    
    @staticmethod
    def aggregate_runs(dataframe):
        
        '''
        Return aggregate information about last runs from each dealer.
        
        '''
        
        temp = dataframe.copy()
        
        wgt_avg_bid_spd = (temp["Bid Spd"] * temp["Bid Size"] / temp["Bid Size"].sum()).sum()
        wgt_avg_ask_spd = (temp["Ask Spd"] * temp["Ask Size"] / temp["Ask Size"].sum()).sum()
        
        tot_bid_size = temp["Bid Size"].sum()
        tot_ask_size = temp["Ask Size"].sum()
           
        bid_rank_3 = list(temp[temp["Bid Size"] > 0].sort_values(by = "Bid Size", ascending = False)["Dealer"][:3])
        bid_rank_3 += [0] * (3 - len(bid_rank_3))

        ask_rank_3 = list(temp[temp["Ask Size"] > 0].sort_values(by = "Ask Size", ascending = False)["Dealer"][:3])
        ask_rank_3 += [0] * (3 - len(ask_rank_3))
        
        n_bids = temp[temp["Bid Size"] > 0]["Dealer"].count()
        n_asks = temp[temp["Ask Size"] > 0]["Dealer"].count()
        
        tot_net_axe = tot_bid_size - tot_ask_size
        tot_gross_axe = tot_bid_size + tot_ask_size
                
        return pd.DataFrame({"Dealer" : ["Aggregate"],
                             "Bid Size" : [tot_bid_size],
                             "Ask Size" : [tot_ask_size],
                             "Bid Spd" : [wgt_avg_bid_spd],
                             "Ask Spd" : [wgt_avg_ask_spd],
                             "Largest Bid" : [bid_rank_3[0]],
                             "2nd Bid" : [bid_rank_3[1]],
                             "3rd Bid" : [bid_rank_3[2]],
                             "Largest Ask" : [ask_rank_3[0]],
                             "2nd Ask" : [ask_rank_3[1]],
                             "3rd Ask" : [ask_rank_3[2]],
                             "n Bids" : [n_bids],
                             "n Asks" : [n_asks],
                             "Net Axe" : [tot_net_axe],
                             "Gross Axe" : [tot_gross_axe]  
                            })
    
    @staticmethod
    def time_series_aggregate_runs(dates, start_time, end_time, isin):
        
        '''
        Return a time series of the aggregate information collected from function msg1_api.aggregate_runs.       
        
        date: string in format "yyyy-mm-dd"
        start_time, end_time: strings in format "Thh:mm:ss" (end_time > start_time)
        isin: string 
        
        '''
    
        dictionary = dict()
    
        for date in dates:
            
            try:
            
                dictionary[date] = msg1_api.aggregate_runs(
                
                    msg1_api.last_runs(
                    
                        msg1_api.clean_runs(
                        
                            msg1_api.get_runs(date = date, 
                                              start_time = start_time, 
                                              end_time = end_time, 
                                              isin = isin))))
                
            except KeyError as e:
                
                print(f"Error on {date}: {e}")
                
                continue
        
        temp = pd.concat(dictionary, axis = 0).reset_index(level = 1, drop = True)
        
        temp["Momentum"] = temp["Net Axe"].diff(periods = 1)
        
        temp.fillna(0, inplace = True)
        
        return temp
    
    @staticmethod
    def multi_bond_time_series_aggregate_runs(dates, start_time, end_time, isins):

        '''
        Return a time series of the aggregate information collected from function msg1_api.aggregate_runs for multiple ISINs.       
        
        date: string in format "yyyy-mm-dd"
        start_time, end_time: strings in format "Thh:mm:ss" (end_time > start_time)
        isins: list of strings 
        
        '''        
        
        all_dates = pd.to_datetime(dates)
        
        dictionary = dict()
        
        for isin in isins:
            
            dictionary[isin] = msg1_api.time_series_aggregate_runs(dates, start_time, end_time, isin).reindex(dates, fill_value = 0)          
                
        columns_to_sum = ['Bid Size', 'Ask Size', 'n Bids','n Asks']

        first_key = next(iter(dictionary))
        
        temp = pd.DataFrame(0, index = dictionary[first_key].index, columns = columns_to_sum)

        for df in dictionary.values():
            
            temp += df[columns_to_sum]
        
        temp["Net Axe"] = temp["Bid Size"] - temp["Ask Size"]
        
        temp["Gross Axe"] = temp["Bid Size"] + temp["Ask Size"]
        
        temp["Momentum"] = temp["Net Axe"].diff(periods = 1)
        
        temp.fillna(0, inplace = True)
        
        return temp            